In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
import math
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText

In [3]:
data = pd.read_csv("train.csv",dtype=object,na_values=str).values

In [4]:
x = np.array(data[:,2:4])
y = np.array(data[:,1])
print(x[0:4])

[['There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV'
  'She left her husband. He killed their children. Just another day in America.']
 ["Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song"
  'Of course it has a song.']
 ['Hugh Grant Marries For The First Time At Age 57'
  'The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.']
 ["Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork"
  'The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.']]


In [5]:
#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [6]:
corpus_raw = []
for i in range(0,x.shape[0]):
    for sent in x[i]:
        if np.nan_to_num(sent) == 0:     # If short description is missing
            corpus_raw.append("NA")      # Necessary, otherwise while feeding training data into NN can cause mismatch
        elif np.nan_to_num(sent) != 0:
            corpus_raw.append(sent)

In [7]:
print(corpus_raw[0:5])
print(np.shape(corpus_raw))
print(type(corpus_raw[24199]))
#words = corpus_raw[0].split()
#print(words)
print(np.nan_to_num(corpus_raw[24199]))

['There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV', 'She left her husband. He killed their children. Just another day in America.', "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song", 'Of course it has a song.', 'Hugh Grant Marries For The First Time At Age 57']
(401664,)
<class 'str'>
NA


In [8]:
# List of words
words2 = []
tokenizer = RegexpTokenizer(r'\w+')
i = 0
for sent in corpus_raw:
    #words.append(sent.split())
    #i = i + 1
    #print(i)
    for word in tokenizer.tokenize(sent):
        words2.append(word.lower())
    
# for i in range(0,len(corpus_raw)):
#     for word in corpus_raw[i].split():
#         if word != '.':       # because we don't want to treat . as a word
#             words.append(word)
#print(type(words))
#words = np.array(words)
#print(type(words))
#print(len(words))
#print(words[0])

Vocab = set(words2)    # so that all duplicate words are removed
vocab = list(Vocab)
# vocab = []
# for j in words:
#     if j not in vocab:
#         vocab.append(j)


word2int = {}
int2word = {}
vocab_size = len(vocab)  # gives the total number of unique words

for i,word in enumerate(vocab):
    word2int[word] = i
    int2word[i] = word

In [9]:
# List of lists of words
words = []
sentences = []
tokenizer = RegexpTokenizer(r'\w+')
i = 0
for sent in corpus_raw:
    #words.append(sent.split())
    #i = i + 1
    #print(i)
    for word in tokenizer.tokenize(sent):
        words.append(word.lower())
    sentences.append(words)
    words = []
# for i in range(0,len(corpus_raw)):
#     for word in corpus_raw[i].split():
#         if word != '.':       # because we don't want to treat . as a word
#             words.append(word)
#print(type(words))
#words = np.array(words)
#print(type(words))
#print(len(words))
#print(words[0])

#Vocab = set(words)    # so that all duplicate words are removed
#vocab = list(Vocab)
# vocab = []
# for j in words:
#     if j not in vocab:
#         vocab.append(j)


# word2int = {}
# int2word = {}
# vocab_size = len(vocab)  # gives the total number of unique words

# for i,word in enumerate(vocab):
#     word2int[word] = i
#     int2word[i] = word

In [10]:
print(len(words))
print(vocab_size)
print(words[0:100])
print(type(vocab))
#Vocab = list(vocab)
#print(vocab[0:100])
# for i in range(0,5):
#     print(i)
# for sent in corpus_raw:
#     print(type(sent))
print(len(word2int))
print(len(int2word))
#print(int2word)
print(len(sentences))
print(sentences[0:5])

0
86718
[]
<class 'list'>
86718
86718
401664
[['there', 'were', '2', 'mass', 'shootings', 'in', 'texas', 'last', 'week', 'but', 'only', '1', 'on', 'tv'], ['she', 'left', 'her', 'husband', 'he', 'killed', 'their', 'children', 'just', 'another', 'day', 'in', 'america'], ['will', 'smith', 'joins', 'diplo', 'and', 'nicky', 'jam', 'for', 'the', '2018', 'world', 'cup', 's', 'official', 'song'], ['of', 'course', 'it', 'has', 'a', 'song'], ['hugh', 'grant', 'marries', 'for', 'the', 'first', 'time', 'at', 'age', '57']]


In [11]:
sentences = np.array(sentences)

In [12]:
model_word2vec = Word2Vec(sentences, size=100, window=5, min_count=0,workers=10,sg=0)
model_word2vec.train(sentences,total_examples=len(sentences),epochs=10)

(49854872, 60996040)

In [13]:
print(model_word2vec)
print(model_word2vec['killing'])
print(model_word2vec.wv.most_similar("shootings"))

Word2Vec(vocab=86718, size=100, alpha=0.025)
[ 0.15428673  0.3032078  -1.1721619   1.874103   -1.5606085  -0.24525835
  2.2021377  -2.475612   -1.7615445   2.326912   -0.34606314 -1.062049
 -2.2459078  -0.15821838 -0.0905036   0.8615143  -2.3459883  -1.5937464
 -0.3024151   0.47964537 -0.29445362  0.11982829  0.3789861   1.4195501
 -0.38800597 -0.37164438  1.7658195  -0.5426802   0.27196807 -0.8482216
  0.53250164 -0.33730203 -0.6293161  -0.3897586   2.6819797  -0.40592122
 -0.14170778 -0.66269076  0.7727368  -2.8266966  -0.35886714 -0.7655429
 -2.1962283  -2.2179437  -2.791443   -1.0361419   1.4445623   0.84580624
 -0.05743724 -0.4303834  -0.47139132  0.29042226 -1.8581903  -1.1161158
 -0.01754724  1.4625732   2.1342294  -0.40597525 -0.37957555  0.5402142
 -0.8409273   0.12632468  0.8405126   1.2006891   2.2413278  -0.64666384
 -0.3393913   0.21993287 -0.9356112   0.19827822 -0.7891616  -1.3792859
  0.51050097 -0.04699465  0.51752055 -0.20769632  0.81789476 -0.4723141
 -2.1866217  -0.

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [14]:
model_word2vec.save("Saved_model_word2vec")

In [15]:
model = Word2Vec.load("Saved_model_word2vec")
print(model)

Word2Vec(vocab=86718, size=100, alpha=0.025)


In [16]:
embeddings = model[model.wv.vocab]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [17]:
print(type(embeddings))
print(embeddings.shape)

<class 'numpy.ndarray'>
(86718, 100)


In [18]:
print(embeddings[2])
print(model['there'])
#print(model['There'])

[-0.01042868  0.02176324 -0.02250957 -0.06258629  0.00782776  0.02794897
  0.00023065 -0.0720012   0.01331906  0.0220496  -0.0013377  -0.02216832
 -0.02165697  0.06010749 -0.0029701   0.03460391 -0.07234558  0.05033163
 -0.00130906  0.07243614  0.05886856 -0.00194149  0.01002414  0.02834561
  0.02269987 -0.01992226 -0.05661791 -0.04028002  0.08232529  0.01287358
 -0.00188548 -0.03678086 -0.15531382  0.00495727  0.00194723  0.01899534
 -0.03533955 -0.00352425 -0.06034484  0.0072311   0.00225596  0.07923514
 -0.02116636 -0.02062715 -0.00707681 -0.02920688  0.08443666 -0.04279945
  0.01290212  0.01215223 -0.04320362 -0.01869603 -0.03856771  0.02556449
 -0.11824536  0.09582601 -0.09653168  0.01564028 -0.0628624   0.03083246
  0.03544985 -0.05076908  0.00566671  0.02384304  0.0216185  -0.03771033
  0.04122637  0.00288024 -0.02996035 -0.0177307   0.06270609 -0.03524997
 -0.00827872 -0.0556549  -0.01649423 -0.04432337  0.03637278  0.02263447
  0.03852658 -0.00167972  0.05346249 -0.01165624  0

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [19]:
# #print(word2int[0])
# for i in range(0,10):
#     print(int2word[i])

In [20]:
#word_vectors = np.zeros([],dtype=float)
#print(words2[0:100])
print(sentences[1])
print(len(sentences[1]))
#print(type(sentences))

['she', 'left', 'her', 'husband', 'he', 'killed', 'their', 'children', 'just', 'another', 'day', 'in', 'america']
13


In [21]:
print(model[sentences[1]])
print(0%2)
print(1%2)
print(model['she'])

[[-0.04204395 -2.184356   -1.8329806  ... -0.6191215   1.1494336
   0.1696423 ]
 [-1.6818107   0.13794908 -1.2052062  ...  1.0178206   1.2658703
   0.9165124 ]
 [ 0.6550478  -0.68891364 -1.1917928  ... -1.6304047   0.697925
   2.8708901 ]
 ...
 [-2.179049    0.0059576  -0.39542943 ... -1.3494959   0.05578793
  -0.8808921 ]
 [ 1.1458899   0.71995765 -1.3256361  ...  2.1202245   0.10713822
   0.65151864]
 [-1.7001518  -1.1210458  -0.04484557 ... -2.0321565   0.22813459
  -0.35970476]]
0
1
[-0.04204395 -2.184356   -1.8329806   2.1544557   3.6301565  -0.9784656
 -2.2373903   2.2429914  -0.64196104 -1.3698995   0.42327255 -1.1105772
 -1.5183218   0.6469478  -0.30586     0.93902755  4.018184    1.2049829
  0.18615219  1.6285034  -0.8934569  -3.2473712   2.0741017  -1.1492478
 -1.8982716  -0.63891333 -0.2758937  -2.1492312   3.7780826   1.3753589
 -0.17512439  1.0000547   0.6103615  -0.10518404  1.2457719  -0.28399262
 -0.6646198  -1.792475    0.5346496   4.1433077  -0.0472825  -3.5070083
  2

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [22]:
X = []
Y = []
temp = []
for i in range(0,len(sentences)):
    for j in range(0,len(sentences[i])):
        temp.append(model[sentences[i][j]])
    if(i%2!=0):
        X.append(temp)
        temp = []
        

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [23]:
print(type(X))
print(np.shape(X))

<class 'list'>
(200832,)


In [24]:
print(len(X[0]))    # Should be sum of no of words in sentences 1 and 2 (Headline and description for first training example)
# Output = 27 (14+13)  Training data ready.

27


In [25]:
# test = []
# test2 = []
# test.append(model[sentences[0][0]])
# print(test)
# test.append(model[sentences[0][1]])
# print(test)
# test2.append(test)
# print(test2)

In [26]:
print(len(X[3443]))
max1 = 0
for i in range(0,len(X)):
    if(len(X[i])>max1):
        max1 = len(X[i])
        pos = i
print(max1)
print(pos)

35
256
58142


In [27]:
print(58142*2)

116284


In [28]:
print(len(sentences[116285]))

254


In [29]:
count = 0
for i in range(0,len(X)):
    if(len(X[i])>100):
        count = count + 1
print(count)

#  Sequence length : Length of each training example
#  Sequence length is varying from 1 to 250, we have to choose a dimension and accordingly all training exapmles would be
#  padded or truncated

219


In [30]:
import keras
from keras.preprocessing.sequence import pad_sequences
X_new = keras.preprocessing.sequence.pad_sequences(sequences=X, maxlen=100, dtype='float32', padding='post', truncating='post', value=0.0)

Using TensorFlow backend.


In [37]:
#print(X[0])
#print(X_new[0])
#print(len(X_new[0]))

100


In [38]:
print(type(X_new))

<class 'numpy.ndarray'>


In [ ]:
np.save("X_new",X_new)

In [ ]:
np.savetxt("X_new_text",X_new)

In [ ]:
###-----------------One-hot labels generation------------------###

In [ ]:
###--------------------------CNN model-------------------------###